# The Problem

In recent years there have been significant reductions in local authority funding,
and budgets have been reduced accordingly.
Cutting spending on preventative measures like early help in schools, quality of life
improvements for elderly and disabled residents, could lead to having to spend
more later to fix more severe problems as a result.
Using a sample spending dataset from a local authority, design a model to test this
theory. (Look for ‘Spend over £500’ open data).
Remember:
* What question are you answering?
* If you had an answer, what would it look like?
* What data would be relevant?
* What are the principal components?

In [19]:
import os
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from natsort import natsorted

# Get the data

- Bristol Council did provide some over £500 spending data as a single file download.

- However, they also provided more data (going back further) as monthly `.csv` files.

- So task 1 was to download and combine all this data.

# Create a web scraper to download and combine .csv files.

- I used the `BeautifulSoup` webscraper to do this.

- The council's website was returning a 403 error when it was being scraped. This was due to it being set up to reject web scrape requests. So we need to set up a fake agent to make the website think the requests are coming from a real user rather than a scraper.

In [ ]:
# dummy header to make the website think a real user is requesting the data
HEADERS = {'User-Agent': 'Mozilla/5.0 (iPad; CPU OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148'}

# URL of the webpage containing CSV files
url = 'https://www.bristol.gov.uk/council-and-mayor/council-spending-and-performance/spending-over-500'

# Download the webpage content
response = requests.get(url, headers=HEADERS)
soup = BeautifulSoup(response.content, 'html.parser')

# By inspecting the webpage in chrome I found that all the links to the
# csv files had a 'type' so we can tell the scraper to look for this type.
# Find all links with a type attribute of 'text/csv' and extract their URLs
links = []
for link in soup.find_all('a', type='text/csv'):
    links.append(urljoin(url, link.get('href')))

In [ ]:
# Download all CSV files and save them in the csv_folder

# set a variable that we can loop through in order to name all the .csv files
i=0

# iterate over the list of links and download each csv file
for link in links:
    try:
        file_response = requests.get(link, headers=HEADERS)
    
        # save the file to the csv_files folder
        with open(f"csv_files original scrape/{str(i)}.csv", "wb") as f:
            f.write(file_response.content)
            i += 1
    except Exception as e:
        print(f"Error downloading CSV file from {link}: {e}")

# clean the csv files

- The files don't share the same formatting (unfotunately!) so we need to clean them up before combining them.

In [4]:
# set the directory containing the CSV files
DIRECTORY = "csv_files/"

In [42]:
# rename the csv files to show month and date.

# the files were downloaded in month order so we can set the start month and year
# and then decrease each in descending order.

start_month = 2
start_year = 2023

# sort the list of files in ascending order (natsort can sort string numbers properly)
wd = os.getcwd()
print(wd)

file_list = os.listdir(DIRECTORY)
file_list = natsorted(file_list)

# iterate over the csv files
for filename in file_list:
    os.rename(DIRECTORY + filename, DIRECTORY + f'{start_year}-{start_month}.csv')
    start_month = start_month - 1

    # A loop to reduce the year when we get to January and then reset the month back to December
    if start_month == 0:
        start_month = 12
        start_year = start_year - 1

/Users/samuelspeller/Documents/GitHub/Bristol-council-spending-analysis


In [6]:
# iterate over the csv files
for filename in os.listdir(DIRECTORY):
    if filename.endswith('.csv'):
        # read the CSV file into a pandas DataFrame 
        df = pd.read_csv(os.path.join(DIRECTORY, filename), encoding='latin', on_bad_lines='warn')

        # dictionary of column names to rename
        rename_columns = {
            'Name': 'Supplier', 
            'Description Line 1': 'Description 1',
            'Description Line 2': 'Description 2',
            'Description Line 3': 'Description 3'
        }
        # rename columns
        for key, value in rename_columns.items():
            if key in df.columns:
                df.rename(columns={key:value}, inplace = True)

        # Some of the data doesn't contain transaction dates so lets at least add
        # the month and year from the file name.
        
        if 'Pay Date' not in df.columns:
            date = os.path.splitext(filename)
            df.insert(2, 'Pay Date', date[0])

        # Some of the files have transaction numbers and some don't.
        # Lets drop all of the transaction/ref columns.
        # Create a list of column names to delete
        del_columns = ['Body', 'Body Name', 'Transaction Number', 'Ref', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 0']
        for name in del_columns:
            if name in df.columns:
                df = df.drop(columns=[name])
        
        # add description 3 column if it's missing
        #if 'Description 3' not in df.columns:
        #    df.insert(5, 'Description 3', ' ' )

        # look for all the files which don't have the correct amount of columns
        if len(df.columns) != 6:
            print(filename)
        
        # save the modified DataFrame back to the CSV file
        df.to_csv(os.path.join(DIRECTORY, filename), index=False)
        

## I want to check that all the csv files have the same column names

- I want to do this as a final check before combining all the `.csv` files.

In [8]:
# Define path to the directory containing CSV files
DIRECTORY = 'csv_files/'
csv_files = sorted(os.listdir(DIRECTORY))

# Read the first CSV file in the list to get the column names
first_file = pd.read_csv(os.path.join(DIRECTORY, csv_files[0]))
column_names = first_file.columns.tolist()

# Loop through the rest of the CSV files and check if they have the same columns
for file in csv_files[1:]:
    next_file = pd.read_csv(os.path.join(DIRECTORY, file))
    if next_file.columns.tolist() != column_names:
        print(f'Column names in {format(file)} do not match')
        exit()

print('all columns match!')

all columns match!


## Combine the csv files
Now we can combine all the csv files so we have one large dataset

In [11]:
# set the directory containing the CSV files
DIRECTORY = "csv_files/"

# create an empty list to store the dataframes
dataframes = []

# loop through the CSV files in the directory and append their dataframes to the list
for filename in os.listdir(DIRECTORY):
    if filename.endswith(".csv"):
        filepath = os.path.join(DIRECTORY, filename)
        df = pd.read_csv(filepath)
        dataframes.append(df)

# concatenate the dataframes in the list into a single dataframe
combined_df = pd.concat(dataframes, ignore_index=True)

# write the combined dataframe to a new CSV file
combined_df.to_csv('bristol_spending_data.csv', index=False)

# There are lots of weird characters present in the dataframe 
- This is probably due to encoding issues.

- Lets remove these characters.

In [18]:
# characters we want to remove
chars_to_remove = ['Â', 'Ã']

# define a function to replace letters with an empty string
def clean_text(x, chars_to_remove):
    if isinstance(x, str):
        for char in chars_to_remove:
            x = x.replace(char, '')
        return x
    else: 
        return x

# read csv
df = pd.read_csv('bristol_spending_data.csv')

# apply the function to each element in the DataFrame using applymap()
df_cleaned = df.applymap(lambda x: clean_text(x, chars_to_remove))
df_cleaned.to_csv('bristol_spending_data_final.csv', index=False)

# Clean and combine complete
We now have a cleaned set of data. 

- All the columns match.
- Transactions with no timestamp have been given one based on the month they occured in (not the best but better than nothing).
- We are now ready to investigate this data.
